# Import Libraries

In [1]:
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

# Data Loading

In [2]:
data = pd.read_csv('final_dataset.csv')
df = data.copy()
df

,author,comment,sentiment,timestamp,like_count,tipe_produk,brand,segment,release_date
0,@rinzia2346,"""batre 5000 mah kerasa kurang awet ga sih di h...",Negative,2024-10-23T05:11:29Z,0,Tecno Spark 20C,Tecno,Entry-Level,24-Jan
1,@LiriklaguKu-x8s,"Bang kok punya ku g ada fitur nfc nya, solusin...",Negative,2024-11-04T10:29:48Z,0,Tecno Spark 20C,Tecno,Entry-Level,24-Jan
2,@encischannel9849,Gue juga pake tekno spark 20c tp ko g ada nfc ya,Negative,2024-09-11T15:30:53Z,0,Tecno Spark 20C,Tecno,Entry-Level,24-Jan
3,@WelderIjo,hp dont play pubg nih,Negative,2024-05-03T12:14:37Z,0,Tecno Spark 20C,Tecno,Entry-Level,24-Jan
4,@edybaskorobaskoro9660,Frame dropnya kok keliatan parah gitu ? Jauh s...,Negative,2024-02-29T18:19:10Z,0,Tecno Spark 20C,Tecno,Entry-Level,24-Jan
...,...,...,...,...,...,...,...,...,...
3317,@leotravel85,"Saya ingin baterai yang lebih besar, bukan pon...",Negative,2024-11-04T09:32:52Z,0,Galaxy S25 Slim,Samsung,Flagship,Mid-2025
3318,@skeletor7708,Berikan saya Samsung Galaxy Ultra 6.1 inci den...,Positive,2024-11-04T09:16:53Z,0,Galaxy S25 Slim,Samsung,Flagship,Mid-2025
3319,@Jz-fj5ki,Samsung BAWA PANEL LAYAR M14 UNTUK S25 ULTRA s...,Negative,2024-11-04T09:02:43Z,0,Galaxy S25 Slim,Samsung,Flagship,Mid-2025
3320,@Jz-fj5ki,"Ponsel tipis adalah yang terburuk, selalu rent...",Negative,2024-11-04T08:58:32Z,0,Galaxy S25 Slim,Samsung,Flagship,Mid-2025


# Feature Engineering

## Balancing Data

> *Balancing* dilakukan agar dataset berada dikondisi yang proposional. *Balancing* dilakukan dengan metode *undersampling* bertujuan untuk menyeimbangkan berdasarkan jumlah kelas data paling sedikit.

In [3]:
# 1. Check class distribution
print("Original Class Distribution:")
print(df['sentiment'].value_counts())

Original Class Distribution:
Negative    1398
Positive    1057
Neutral      865
Name: sentiment, dtype: int64


In [4]:
# 2. Identify the minimum class size
min_count = df['sentiment'].value_counts().min()

In [5]:
# 3. Undersample majority classes
undersampled_data = df.groupby('sentiment').apply(lambda x: x.sample(min_count)).reset_index(drop=True)

In [6]:
# 4. Check new class distribution
print("Undersampled Class Distribution:")
print(undersampled_data['sentiment'].value_counts())

Undersampled Class Distribution:
Negative    865
Neutral     865
Positive    865
Name: sentiment, dtype: int64


In [7]:
# 5. View the final dataset
print("Final Undersampled Dataset:")
undersampled_data

Final Undersampled Dataset:


,author,comment,sentiment,timestamp,like_count,tipe_produk,brand,segment,release_date
0,@ianembenk9459,Kenapa lebih bagus Redmi Note 14 Pro Plus desa...,Negative,2024-10-13T15:00:01Z,1,Xiaomi 14T,Xiaomi,Flagship,24-Sep
1,@c.c.churaa2573,Gw coba beli ini hp minus kamera ultrawide,Negative,2024-08-20T00:46:25Z,0,Realme 13 Series,Realme,Mid-Range,2025
2,@ninjasamurai6094,sebel xiaomi after sales service nya jelek ban...,Negative,2024-11-08T14:38:18Z,0,Xiaomi 15,Xiaomi,Flagship,2025
3,@AkuCayang,Asik pink di mukanya ilang ð kek tang di co...,Negative,2024-08-21T12:16:30Z,0,Galaxy S24 Ultra,Samsung,Flagship,18-Jan-24
4,@crystalsnow3789,Tetap red magic6,Negative,2024-01-25T18:03:45Z,0,OPPO Find X7,OPPO,Flagship,2025
...,...,...,...,...,...,...,...,...,...
2590,@oscarardiano4215,Hp impian,Positive,2024-10-26T03:22:16Z,0,Vivo X100 Pro,Vivo,Flagship,2024
2591,@ekadiasputramahendra4056,Masih prefer ke realme 12+ 5G ð,Positive,2024-03-21T10:24:38Z,2,Realme 12 5G,Realme,Mid-Range,24-Feb
2592,@purimutia2192,Cocok kayaknya buat ortu utk penggunaan komuni...,Positive,2024-11-14T11:57:41Z,0,Galaxy A06,Samsung,Entry-Level,24-Nov
2593,@mikamasamoto5748,"Habis beli ...serius , klo liat langsung desai...",Positive,2024-10-21T13:56:25Z,0,Xiaomi 14T,Xiaomi,Flagship,24-Sep


## Text Preprocessing

Berdasarkan EDA yang telah dilakukan, terdapat beberapa tindakan yang dapat dilakukan untuk membersihkan teks sehingga dapat meningkatkan performa dalam **Modeling**, yaitu
- *Case folding*: membuat semua kata dalam kondisi *lower* agar mudah di analisis. Jika terdapat perbedaan *Case folding* dana kata tersebut sama, maka akan dianggap 2 kata yang berbeda oleh mesin.
- *Mention removal*: menghapus simbol @ yang biasa digunakan dalam memanggil suatu akun.
- *Hashtags removal*: menghapus # yang biasa digunakan sebagai keyword dalam suatu narasi kalimat.
- *Newline removal (\n)*: menghapus kondisi baris yang terdapat *Newline* yang mengakibatkan mesin tidak dapat bekerja secara efisien dalam menganalisis.
- *Whitespace removal*: menghapus suatu baris yang memiliki *space* yang besar.
- *URL removal*: menghapus suatu baris yang yang mencantumkan link website.
- *Non-letter removal*: menghapus karakter yang berupa simbol seperti yang tadi sempat muncul secara jelas ketika EDA.
- *replace slang*: mengganti istilah kata gaul dan singkatan menjadi kata dasar. 
- *Stopwords removal*: menghapus kata sifat dan subjek yang tidak dibutuhkan dalam teks sehingga meningkatkan performa ketika *modeling*.
- *Lemmatizing*: mengubah kata kerja berimbuhan ke dalam bentuk dasar.

In [8]:
# mengganti istilah gaul dan singkatan kata
slang_dict = {
    "gw": "saya",
    "mau": "ingin",
    "ni": "ini",
    "aja": "saja",
    "gak": "tidak",
    "bgt": "sangat",
    'klo': 'kalau',
    'bgs': 'bagus',
    'masi': 'masih',
    'msh': 'masih',
    'lom': 'belum',
    'blm': 'belum',
    'ap': 'apa',
    'brg':'barang',
    'ad': 'ada',
    'blom': 'belum',
    'kebli': 'kebeli',
    'tp': 'tapi',
    'org': 'orang',
    'tdk': 'tidak',
    'yg': 'yang',
    'kalo': 'kalau',
    'sy': 'saya',
    'ni':'ini',
    'bng': 'abang',
    'bg': 'abang',
    'fto': 'foto',
    'spek': 'spesifikasi',
    'cm': 'cuma',
    'jg': 'juga',
    'pd': 'pada',
    'skrg': 'sekarang',
    'ga': 'tidak',
    'gk': 'tidak',
    'bgt': 'sangat',
    'batre': 'baterai',
    'gue': 'saya',
    'dpt': 'dapat',
    'kek': 'seperti',
    'mna': 'mana',
    'mnding': 'mending',
    'mend': 'mending',
    'dr': 'dari',
    'sma': 'sama',
}

def replace_slang(text, slang_dict):
    words = text.split()
    return ' '.join([slang_dict.get(word, word) for word in words])

In [9]:
# kata-kata dalam teks yang akan dikeluarkan dari list stopword

kata = ['baru', 'lama', 'sama', 'tapi', 'tidak', 'dari',
        'belum', 'bagi', 'mau', 'masalah', 'kecil', 
        'jumlah', 'cara', 'apa', 'ada', 'seperti', 
        'cuma', 'sekarang', 'ingin', 'besar', 'bisa', 'wah', 
        'mirip', 'lah', 'saja', 'buat', 'waktu', 'masih', 
        'daripada', 'banyak', 'bakal', 'sangat', 'tanpa', 'mana']

In [10]:
# define stopword
stpwds_id = list(set(stopwords.words('indonesian')))
for i in kata:
    stpwds_id.remove(i) # menghapus kata-kata yang ada di stopword sehingga kata tersebut dapat digunakan dalam teks

# Create WordNetLemmatizer object
lemmatizer = WordNetLemmatizer()

In [11]:
# Create A Function for Text Preprocessing

def text_preprocessing(text):
    # Case folding
    text = text.lower()

    # Mention removal
    text = re.sub("@[A-Za-z0-9_]+", " ", text)

    # Hashtags removal
    text = re.sub("#[A-Za-z0-9_]+", " ", text)

    # Newline removal (\n)
    text = re.sub(r"\\n", " ",text)

    # Whitespace removal
    text = text.strip()

    # URL removal
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"www.\S+", " ", text)

    # Non-letter and number removal (such as emoticon, symbol (like μ, $, 兀), etc
    text = re.sub("[^A-Za-z0-9_\s']", " ", text)
    
    # mengganti istilah gaul dan singkatan kata
    text = replace_slang(text, slang_dict)

    # Tokenization
    tokens = word_tokenize(text)

    # Stopwords removal
    tokens = [word for word in tokens if word not in stpwds_id]

    # Lemmatizing
    tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens] # cuma kata kerja saja yang diubah ke bentuk dasar

    # Combining Tokens
    text = ' '.join(tokens)

    return text

In [12]:
# Applying Text Preprocessing to the Dataset

undersampled_data['text_processed'] = undersampled_data['comment'].apply(lambda x: text_preprocessing(x))
undersampled_data

,author,comment,sentiment,timestamp,like_count,tipe_produk,brand,segment,release_date,text_processed
0,@ianembenk9459,Kenapa lebih bagus Redmi Note 14 Pro Plus desa...,Negative,2024-10-13T15:00:01Z,1,Xiaomi 14T,Xiaomi,Flagship,24-Sep,bagus redmi note 14 pro plus desain nya
1,@c.c.churaa2573,Gw coba beli ini hp minus kamera ultrawide,Negative,2024-08-20T00:46:25Z,0,Realme 13 Series,Realme,Mid-Range,2025,coba beli hp minus kamera ultrawide
2,@ninjasamurai6094,sebel xiaomi after sales service nya jelek ban...,Negative,2024-11-08T14:38:18Z,0,Xiaomi 15,Xiaomi,Flagship,2025,sebel xiaomi after sales service nya jelek ban...
3,@AkuCayang,Asik pink di mukanya ilang ð kek tang di co...,Negative,2024-08-21T12:16:30Z,0,Galaxy S24 Ultra,Samsung,Flagship,18-Jan-24,asik pink mukanya ilang seperti tang contohin ...
4,@crystalsnow3789,Tetap red magic6,Negative,2024-01-25T18:03:45Z,0,OPPO Find X7,OPPO,Flagship,2025,red magic6
...,...,...,...,...,...,...,...,...,...,...
2590,@oscarardiano4215,Hp impian,Positive,2024-10-26T03:22:16Z,0,Vivo X100 Pro,Vivo,Flagship,2024,hp impian
2591,@ekadiasputramahendra4056,Masih prefer ke realme 12+ 5G ð,Positive,2024-03-21T10:24:38Z,2,Realme 12 5G,Realme,Mid-Range,24-Feb,masih prefer realme 12 5g
2592,@purimutia2192,Cocok kayaknya buat ortu utk penggunaan komuni...,Positive,2024-11-14T11:57:41Z,0,Galaxy A06,Samsung,Entry-Level,24-Nov,cocok kayaknya buat ortu utk penggunaan komuni...
2593,@mikamasamoto5748,"Habis beli ...serius , klo liat langsung desai...",Positive,2024-10-21T13:56:25Z,0,Xiaomi 14T,Xiaomi,Flagship,24-Sep,habis beli serius liat langsung desain nya cak...


## Label Encoding

> Label encoding bertujuan untuk mengubah values ke bentuk diskrit sehingga memudahkan dalam *modeling*.

In [13]:
# Display Target

undersampled_data.sentiment.unique()

array(['Negative', 'Neutral', 'Positive'], dtype=object)

In [14]:
# Change Target into Number

undersampled_data['target'] = undersampled_data['sentiment'].replace({'Negative' : 0, 'Positive' : 1, 'Neutral': 2})

## Data Splitting

> Data Splitting bertujuan memudahkan model ketika berlatih hingga diuji.

In [15]:
# Data Splitting

X_train_val, X_test, y_train_val, y_test = train_test_split(undersampled_data.text_processed,
                                                    undersampled_data.target,
                                                    test_size=0.15,
                                                    random_state=20,
                                                    stratify=undersampled_data.target)

X_train, X_val, y_train, y_val = train_test_split(X_train_val,
                                                  y_train_val,
                                                  test_size=0.10,
                                                  random_state=20,
                                                  stratify=y_train_val)

print('Train Size : ', X_train.shape)
print('Val Size   : ', X_val.shape)
print('Test Size  : ', X_test.shape)

Train Size :  (1984,)
Val Size   :  (221,)
Test Size  :  (390,)


## Missing Values

> Membersihkan *missing values* pada dataset bertujuan untuk mengoptimalkan *modeling* sehingga ketika dievaluasi menghasilkan nilai metrik yang tinggi. *Missing values* dilakukan setelah text preprocessing untuk menghindari timbulnya cell yang kosong atau duplikat setelah dilakukan pembersihan kata dalam teks.

In [16]:
# periksa missing values sebelum di handle
X_train.isnull().sum()

0

In [17]:
# periksa duplikasi sebelum di handle
X_train.duplicated().sum()

41

In [18]:
# drop data duplikat
X_train = X_train.drop_duplicates()

In [19]:
# periksa duplikasi setelah di handle
X_train.duplicated().sum()

0